# Comparison between patients of PD

In this notebook we will make a comparison for PD with early and late onset, taking into account motor and non motor symptoms.

In [ ]:
def get_library_path()->str:

    cwd = os.getcwd()
    lst = cwd.split(os.sep)[:-1]

    path_lib = f'{os.sep}'

    for elem in lst:
        path_lib = os.path.join(path_lib, elem)

    return path_lib

# imports

import os
import sys

path_to_lib = get_library_path()

sys.path.append(path_to_lib)
import luxgiant_clinical.TwoCatAnalysis as two
from luxgiant_clinical.Helpers import categories_recoder

import pandas as pd

In [ ]:
# load data

folder_path = os.path.join(path_to_lib, 'data/source')

df = pd.read_csv(
    os.path.join(folder_path, 'cleaned_file.csv'), 
    low_memory=False
)

In [ ]:
# filter data to keep only patients with age at onset information

mask_patients= (df['Status']=='Patient')
mask_age     = (~df['agecat_1'].isnull()) 

df_cases = df[mask_patients & mask_age].reset_index(drop=True)
del df

df_cases.shape

In [ ]:
motor_at_onset = {
    "initial_motor_symptom_s___2"   : "Rest tremor" ,
    "initial_motor_symptom_s___12"  : "Bradykinesia" ,
    "initial_motor_symptom_s___13"  : "Rigidity",
    "initial_motor_symptom_s___3"   : "Postural/action",
    "initial_motor_symptom_s___8"   : "Reduced dexterity",
    "initial_motor_symptom_s___9"   : "Microphagia",
    "initial_motor_symptom_s___11"  : "Reduced arm swing",
    "initial_motor_symptom_s___16"  : "Gait disorder",
    "initial_motor_symptom_s___14"  : "Freezing",
    "initial_motor_symptom_s___15"  : "Falls",
}
variables1 = ['participant_id', 'agecat_1'] + list(motor_at_onset.keys())

nonmotor_at_onset = {
    "initial_non_motor_symptom___18": "Constipation",
    "initial_non_motor_symptom___8" : "Insomnia",
    "initial_non_motor_symptom___4" : "RBD",
    "initial_non_motor_symptom___4" : "Restless Leg Syndrome",
    "initial_non_motor_symptom___20": "Urinary urgency",
    "initial_non_motor_symptom___13": "Hyperhidrosis",
    "initial_non_motor_symptom___10": "Anosmia/hyposmia",
    "initial_non_motor_symptom___2" : "Cognitive symptoms",
}
variables2 = ['participant_id', 'agecat_1'] + list(nonmotor_at_onset.keys())

motor_current = {
    "current_motor_symptom_s___2"   : "Rest tremor",
    "current_motor_symptom_s___3"   : "Postural/action tremor",
    "current_motor_symptom_s___14"  : "Bradykinesia",
    "current_motor_symptom_s___4"   : "Rigidity",
    "current_motor_symptom_s___15"  : "Gait disorder",
    "current_motor_symptom_s___16"  : "Falls",
    "current_motor_symptom_s___17"  : "Freezing",
    "current_motor_symptom_s___18"  : "Dyskinesias",
    "current_motor_symptom_s___19"  : "Motor fluctuation",
}
variables3 = ['participant_id', 'agecat_1'] + list(motor_current.keys())

nonmotor_current = {
    "current_non_motor_symptom___3" : "Psychosis",
    "current_non_motor_symptom___5" : "Depression",
    "current_non_motor_symptom___6" : "REM sleep behaviour disorder",
    "current_non_motor_symptom___7" : "Restless leg syndrome",
    "current_non_motor_symptom___9" : "Anosmia/hyposmia",
    "current_non_motor_symptom___2" : "Cognitive impairment",
    "current_non_motor_symptom___10": "Excessive daytime sleepness",
    "current_non_motor_symptom___11": "Insomnia",
    "current_non_motor_symptom___14": "Orthostasis",
    "current_non_motor_symptom___16": "Hyperhidrosis",
    "current_non_motor_symptom___17": "Seborrhea",
    "current_non_motor_symptom___20": "Constipation",
    "current_non_motor_symptom___22": "Urinary incontinence"
}
variables4 = ['participant_id', 'agecat_1'] + list(nonmotor_current.keys())

groups = ['Age at onset <50 years', 'Age at onset >=50 years']

In [ ]:
stats = [
    {'n' : list(motor_at_onset.values())},
    {'n' : list(nonmotor_at_onset.values())},
    {'n' : list(motor_current.values())},
    {'n' : list(nonmotor_current.values())}
]

In [ ]:
df_split = [
    df_cases[variables1].copy().rename(columns=motor_at_onset),
    df_cases[variables2].copy().rename(columns=nonmotor_at_onset),
    df_cases[variables3].copy().rename(columns=motor_current),
    df_cases[variables4].copy().rename(columns=nonmotor_current),
]

In [ ]:
for k in range(len(df_split)):
    df_split[k] = categories_recoder(df_split[k], stats[k]['n'], {"Checked":1, "Unchecked":0})
    df_split[k] = categories_recoder(df_split[k], ["agecat_1"], {"<50":'Age at onset <50 years', ">=50":'Age at onset >=50 years'})

In [ ]:
dfs = []
for k in range(len(df_split)):
    dfs.append(
        two.report_proportion(
            data_df    =df_split[k],
            variables  =stats[k]['n'],
            groups     =groups,
            grouping_by='agecat_1',
        )
    )

dfs = [two.final_formatter(df, groups) for df in dfs]

In [ ]:
def dummy_frame(subheader:str, columns:list)->pd.DataFrame:

    temp = pd.DataFrame(columns=columns)
    temp.loc[0] = [subheader] + ['']*len(columns[1:])

    return temp

In [ ]:
subheaders =[
    'Motor symptoms at onset', 'Non-motor symptoms at onset', 
    'Current motor symptoms', 'Current non-motor symptoms'
]

dfs = [
    pd.concat(
        [dummy_frame(subheader=subheaders[k], columns=dfs[k].columns), dfs[k]]) for k in range(len(dfs)
    )
]

In [ ]:
df = pd.concat(dfs, ignore_index=True)
df

In [ ]:
# save result in csv file

df.to_csv(os.path.join(path_to_lib, 'data/auxiliar/new_aux_table_6.csv'), index=False)